In [ ]:
# Normal stack of pandas, numpy, matplotlib and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

import os
import json

# Factual API
from factual import Factual # https://github.com/Factual/factual-python-driver
from factual.utils import circle

%matplotlib inline

Factual API TODOs in order to make it work on your localhost:
 1. run in terminal "pip install factual-api" without ""
 2. sign up for a Factual API key here: https://www.factual.com/contact/new#free_api_access
 3. Set up your key and secret as environment variables - FACTUAL_API_KEY and FACTUAL_API_SECRET respectively
    
    3.1. On mac you just open .bash_profile and add two lines: 
     
     export FACTUAL_API_KEY=your_key
     
     export FACTUAL_API_SECRET=your_secret
     
    3.2. Restart shell and run jupyter notebook in this newly restarted shell (terminal)
     
 4. ...profit ;)

In [ ]:
# You can verify that your env variables are ok with:
# print(os.environ['FACTUAL_API_KEY'])
# print(os.environ['FACTUAL_API_SECRET'])

In [ ]:
factual = Factual(os.environ['FACTUAL_API_KEY'], os.environ['FACTUAL_API_SECRET'])

In [ ]:
places = factual.table('places-ch')
places.schema()

In [ ]:
places.data()

In [ ]:
the_great_escape = places.search('The Great Escape').data()
print(json.dumps(the_great_escape, indent=2))

In [ ]:
#Coffe near EPFL
epfl_coffe = places.geo(circle(46.522773,6.63313, 80)).offset(100).limit(50).data()
print(json.dumps(epfl_coffe, indent=2))

In [ ]:
len(epfl_coffe)

LAUSANE MVP TODOs:
 - Create a dict with coordinates and radius for Lausanne
 - Write up a script scraping API across those coordinates AND
 - write up a system to store scrapped coordinates and untag them from future scrapping


_________
General TODOs:

 - We gotta figure out how we would like to divide Switzerland into small pieces (squares or circles) such that we will be able to extract all spots provided by factual without actually hitting its limit ("3 Page limit=50 is the maximum number of rows that can be returned in a single request using the limit parameter. Row limit=500 is the maximum depth a request can page into using (e.g., offset + limit).").
 - Since we wont be able to squeeze it in 10 000 places/day we also need to figure out a way to run script on multiple machines (using few API-KEYS) and on the course of multiple days.